# Clustering of roads

This notebook contains the code for the clustering of the roads in Milan using different features, such as K_road (source and dest), betweenness and VOC. It also contains the code of the clustering analysis of the different clusters.

In [ ]:
import sumolib
import pandas as pd
import igraph as ig
import numpy as np
import scipy
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import ConvexHull
from result_utils import *

#### parameters

In [ ]:
city = 'Milano_big'
fold_prefix = 'baseline'
p_kroad = 75
p_bc = 75

# road network path
road_network_path = "../data/road_net/"+city+"/"+city+"_road_network.net.xml"

# road-edge map
path_road_edge_mapping = '../data/road_net/'+city+'/'+city+'_road_edge_map.csv'

# experiment results
folder_experiments = '../data/simulations/'+city+'/'+fold_prefix+'/sumo_out/'

# output paths
path_results = "../data/simulations/"+city+"/"+fold_prefix+"/results/"
path_plots = "../data/simulations/"+city+"/"+fold_prefix+"/plots/"

## 1. K_road aggregation and betweenness centrality

Load k_road origin and destination

In [ ]:
with open(path_results+'kroad_O_'+fold_prefix+'.json', 'r') as f:
    k_road_o = json.load(f)
with open(path_results+'kroad_D_'+fold_prefix+'.json', 'r') as f:
    k_road_d = json.load(f)

In [ ]:
road_edge_map = pd.read_csv(path_road_edge_mapping)

In [ ]:
k_road_o_df = pd.DataFrame(columns=['edge_id', 'k_road_o'])
k_road_o_df['edge_id'] = k_road_o.keys()
k_road_o_df['k_road_o'] = k_road_o.values()

k_road_d_df = pd.DataFrame(columns=['edge_id', 'k_road_d'])
k_road_d_df['edge_id'] = k_road_d.keys()
k_road_d_df['k_road_d'] = k_road_d.values()

k_road_od_df = pd.merge(k_road_o_df, k_road_d_df, on='edge_id')

In [ ]:
# k_road computed aggregating by road and with weighted average on length

k_road_df = pd.merge(k_road_od_df, road_edge_map, on='edge_id', how='left')
weighted_avg = lambda x: np.average(x, weights=k_road_df.loc[x.index, "edge_len"])
k_road_df = k_road_df.groupby(by=['road']).agg({'k_road_o': weighted_avg, 'k_road_d': weighted_avg}).reset_index()

Load betweenness centrality

In [ ]:
with open(path_results+'bc_igraph_'+fold_prefix+'.json', 'r') as f:
    edge_bc_map = json.load(f)

In [ ]:
# Normalize bc
val = list(edge_bc_map.values())
max_val = np.max(val)
min_val = np.min(val)
for k,v in edge_bc_map.items():
    edge_bc_map[k] = (v - min_val)/(max_val - min_val)

In [ ]:
bc_edge_df = pd.DataFrame()
bc_edge_df['edge_id'] = edge_bc_map.keys()
bc_edge_df['bc'] = edge_bc_map.values()

In [ ]:
bc_df = pd.merge(bc_edge_df, road_edge_map, on='edge_id', how='left')
weighted_avg = lambda x: np.average(x, weights=bc_df.loc[x.index, "edge_len"])
bc_df = bc_df.groupby(by=['road']).agg({'bc': weighted_avg}).reset_index()

In [ ]:
corr_df = pd.merge(bc_df, k_road_df, on='road')

In [ ]:
del bc_df, bc_edge_df, edge_bc_map, val
del k_road_d, k_road_d_df, k_road_o, k_road_o_df, k_road_od_df, k_road_df

## 2. VOC

Compute the Volume-Over-Capacity

In [ ]:
dict_exps = create_dict_exps(folder_experiments, 'baseline')

In [ ]:
# Aggregate all experiments results and compute the mean and the std of the 'total_of' column.
# It returns a dictionary with keys = roadnames and list [mean, std].

def create_dict_total_per_edge(dict_exps, folder_experiments, main_experiment_name, total_of):
    dict_total = {}
    for exp_id, exp_folder_name in dict_exps[main_experiment_name].items():
        exp_df = pd.read_csv(folder_experiments+exp_folder_name+"/edge_measures.csv")
        
        for ind, row in exp_df.iterrows():
            if row['edge_id'] in dict_total:
                dict_total[row['edge_id']].append(row[total_of])
            else:
                dict_total[row['edge_id']] = [row[total_of]]
    
    list_df = []
    for edge, total in dict_total.items():
        list_df.append([edge, np.array(total).mean(), np.array(total).std()])
    df = pd.DataFrame(list_df, columns=['edge_id', 'mean', 'std'])
        
    return df

In [ ]:
df_edge_volume = create_dict_total_per_edge(dict_exps, folder_experiments, 'baseline', 'total_v_edge')

In [ ]:
def compute_edge_capacity(sumo_edges):

    conversion_factor = 2.2369362912
    q = 0.5

    edge_capacity = {}

    for edge in sumo_edges:

        speed_m_s = edge.getSpeed()
        sl = speed_m_s*conversion_factor

        num_lanes = edge.getLaneNumber()

        # when the speed limit of a road segment sl≤45, it is defined as an arterial road
        if sl <= 45:
            capacity = 1900*num_lanes*q
        # when the speed limit of a road segment 45<sl<60, it is defined as a highway
        elif 45<sl<60:
            capacity = (1000+20*sl)*num_lanes
        # when the speed limit of a road segment sl ≥60, it is defined as a freeway
        elif sl>=60:
            capacity = (1700+10*sl)*num_lanes

        edge_capacity[edge.getID()] = capacity
        
    return edge_capacity

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

In [ ]:
df_edge_capacity = compute_edge_capacity(road_network.getEdges())
df_edge_capacity = pd.DataFrame(df_edge_capacity.items(), columns=['edge_id', 'capacity'])

In [ ]:
df_voc = pd.merge(df_edge_volume, df_edge_capacity, on='edge_id')

In [ ]:
df_voc['voc'] = df_voc['mean']/df_voc['capacity']

In [ ]:
road_edge_map = pd.read_csv(path_road_edge_mapping)

In [ ]:
df_voc = pd.merge(df_voc, road_edge_map, on='edge_id', how='left')

In [ ]:
weighted_avg = lambda x: np.average(x, weights=df_voc.loc[x.index, "edge_len"])
df_voc = df_voc.groupby(by=['road']).agg({'voc': weighted_avg}).reset_index()

In [ ]:
corr_df = pd.merge(corr_df, df_voc, on='road')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(7,9))
fig.suptitle('Milano_big k_road - VOC correlation', fontweight='bold', y=0.93)

axs[0].scatter(corr_df['k_road_o'], corr_df['voc'], s=1)
axs[0].set_title('k_road origin', fontsize=10)
axs[0].set_xticks(np.arange(0,48,2))
#axs[0].set_yscale('log')
axs[0].set_xlabel('k_road')
axs[0].set_ylabel('VOC')

axs[1].scatter(corr_df['k_road_d'], corr_df['voc'], s=1)
axs[1].set_title('k_road destination', fontsize=10)
axs[1].set_xticks(np.arange(0,30,2))
#axs[1].set_yscale('log')
axs[1].set_xlabel('k_road')
axs[1].set_ylabel('VOC')

plt.savefig(path_plots+'k_road_voc_corr.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
print('K_road_o - VOC Pearson corr: '+str(scipy.stats.pearsonr(corr_df['k_road_o'], corr_df['voc']).statistic))
print('K_road_d - VOC Pearson corr: '+str(scipy.stats.pearsonr(corr_df['k_road_d'], corr_df['voc']).statistic))

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

ax.scatter(corr_df['bc'], corr_df['voc'], s=1)
ax.set_title('Betweenness centrality - VOC correlation', fontsize=10)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('bc')
ax.set_ylabel('VOC')

plt.savefig(path_plots+'bc_voc_corr.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
print('Betweenness - VOC Pearson corr: '+str(scipy.stats.pearsonr(corr_df['bc'], corr_df['voc']).statistic))

## 3. Correlation with CO2


In [ ]:
# Aggregate all experiments results and compute the mean and the std of the 'total_of' column.
# It returns a dictionary with keys = roadnames and list [mean, std].

def create_dict_total_per_road(dict_exps, folder_experiments, main_experiment_name, total_of):
    dict_total = {}
    for exp_id, exp_folder_name in dict_exps[main_experiment_name].items():
        exp_df = pd.read_csv(folder_experiments+exp_folder_name+"/road_measures.csv")
        
        for ind, row in exp_df.iterrows():
            if row['road'] in dict_total:
                dict_total[row['road']].append(row[total_of])
            else:
                dict_total[row['road']] = [row[total_of]]
    
    list_df = []
    for road, total in dict_total.items():
        list_df.append([road, np.array(total).mean(), np.array(total).std()])
    df = pd.DataFrame(list_df, columns=['road', 'mean', 'std'])
        
    return df


In [ ]:
df_co2_road = create_dict_total_per_road(dict_exps, folder_experiments, 'baseline', 'total_co2')
df_co2_len = pd.merge(road_edge_map.groupby('road')['edge_len'].sum(), df_co2_road, on=['road'])
df_co2_len['co2'] = df_co2_len['mean']/df_co2_len['edge_len']
df_co2_len['co2_std'] = df_co2_len['std']/df_co2_len['edge_len']

In [ ]:
corr_df = pd.merge(corr_df, df_co2_len[['road', 'co2', 'co2_std']], on=['road'])

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

ax.scatter(corr_df['voc'], corr_df['co2'], s=1)
ax.set_title('VOC - CO2 correlation', fontsize=10)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('VOC')
ax.set_ylabel('CO2')

plt.savefig(path_plots+'co2_voc_corr.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
print('VOC - CO2 Spearman corr: '+str(scipy.stats.spearmanr(corr_df['voc'], corr_df['co2']).correlation))

## 4. Add type of road

Add the type of the road obtained from a split using the K_road and the betweenness centrality. It used for the comparison with the clustering technqiues.

In [ ]:
def split_type_road(df, k_road_percentile, bc_percentile, origin=True):
    types = []
    if origin:
        k_road = 'k_road_o'
    else:
        k_road = 'k_road_d'
    k_road_high = np.percentile(corr_df[k_road], p_kroad)
    bc_high = np.percentile(corr_df['bc'], p_bc)

    for ids, row in corr_df.iterrows():
        if row[k_road] >= k_road_high and row['bc'] >= bc_high:
            types.append('connector')
        elif row[k_road] < k_road_high and row['bc'] >= bc_high:
            types.append('peripheral')
        elif row[k_road] >= k_road_high and row['bc'] < bc_high:
            types.append('attractor')
        else:
            types.append('local')

    return types

In [ ]:
corr_df['y_o'] = split_type_road(corr_df, p_kroad, p_bc, origin=True)
corr_df['y_d'] = split_type_road(corr_df, p_kroad, p_bc, origin=False)

In [ ]:
colors = {'connector':'red', 'attractor':'orange', 'peripheral':'green', 'local':'grey'}

In [ ]:
clust_labels = pd.read_csv(path_results+'road_clust_map.csv')
corr_df = pd.merge(corr_df, clust_labels, on=['road'])

In [ ]:
colors_clust = {'HF': '#de8f05', 'HE': '#029e73', 'LF':'#d55e00', 'LE':'#949494'}

## 5. Clustering

Compute the clustering using the different features and the different clustering techniques.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5,6))
fig.suptitle('Milano_big k_road - betwenness correlation', fontweight='bold', y=0.93)

#axs[0].scatter(corr_df['k_road_o'], corr_df['bc'], c=corr_df['y_o'].map(colors), s=1)
axs[0].scatter(corr_df['k_road_o'], corr_df['bc'], c=corr_df['clust_label'].map(colors_clust), s=1)
axs[0].set_xticks(np.arange(0,48,2))
axs[0].set_yscale('log')
axs[0].set_xlabel('k_road_o')
axs[0].set_ylabel('bc')

#axs[1].scatter(corr_df['k_road_d'], corr_df['bc'], c=corr_df['y_d'].map(colors), s=1)
axs[1].scatter(corr_df['k_road_d'], corr_df['bc'], c=corr_df['clust_label'].map(colors_clust), s=1)
axs[1].set_xticks(np.arange(0,30,2))
axs[1].set_yscale('log')
axs[1].set_xlabel('k_road_d')
axs[1].set_ylabel('bc')

plt.show()

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, accuracy_score, jaccard_score, f1_score
from sklearn.metrics.cluster import adjusted_rand_score, fowlkes_mallows_score
from sklearn.preprocessing import MinMaxScaler
from kneed import KneeLocator

In [ ]:
X = corr_df[['k_road_o', 'k_road_d', 'bc', 'voc']].values
y_o = corr_df['y_o'].values
y_d = corr_df['y_d'].values

In [ ]:
scaler = MinMaxScaler().fit(X)

In [ ]:
## scaler       inertia (4)     silhouette (4)      calinski (4)        behavior
## unscaled     30000 (40k)     0.47 (0.50)         8000 (9k)           6 clusters, vertically divided by k_road
## minmax       55 (60)         0.44 (0.45)         4800 (5100)         5 clusters, vertically divided by k_road
## standard     8000 (10k)      0.52 (0.55)         3700 (3700)         5 clusters, vertically divided by k_road, but low bc separated
## robust       40000 (60k)     0.66 (0.67)         5100 (4700)         6 clusters, horizontally divided by betweenness for low k_road

### 5.1 K-means

#### 5.1.1 Grid search k-means

In [ ]:
# Each clustering repeated 20 times, best number of clusters selected with elbow method
## K-means params       time    inertia     silhouette      calinski
## k-means++ - lloyd    21m     [4,5]       [4,5]           [3,4,5]
## k-means++ - elkan    22m     [4,5]       [4,5,6]         [3,4,5]
## random - lloyd       27m     [4,5]       [4,5]           [3,4,5]

In [ ]:
clusters = np.arange(2,11)

In [ ]:
ssd_score = {} #inertia (sum of squared distances) --> the lower the better --> elbow method
sil_score = {} #silhouette --> 1 better, -1 worse --> elbow method
ch_score = {} #calinski_harabasz index --> the higher the better --> elbow method

In [ ]:
X_scaled = scaler.transform(X)
for i in clusters:
    ssd_tmp = []
    sil_tmp = []
    ch_tmp = []
    for j in range(0, 20):
        clf = KMeans(n_clusters=i, init='k-means++', n_init=5, algorithm='lloyd').fit(X_scaled)
        ssd_tmp.append(clf.inertia_)
        sil_tmp.append(silhouette_score(X_scaled, clf.labels_))
        ch_tmp.append(calinski_harabasz_score(X_scaled, clf.labels_))

    ssd_score[i] = np.array(ssd_tmp).mean()
    sil_score[i] = np.array(sil_tmp).mean()
    ch_score[i] = np.array(ch_tmp).mean()

In [ ]:
plt.plot(ssd_score.keys(), ssd_score.values(), '-o', ms=3)
elbow = KneeLocator(list(ssd_score.keys()), list(ssd_score.values()), curve='convex', direction='decreasing').knee
plt.axvline(elbow, c='tab:red', ls='--')
plt.xticks(clusters)
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
#plt.savefig(path_plots+'kmeans_inertia.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
plt.plot(sil_score.keys(), sil_score.values(), '-o', ms=3)
elbow = KneeLocator(list(ssd_score.keys()), list(ssd_score.values()), curve='convex', direction='decreasing').knee
plt.axvline(elbow, c='tab:red', ls='--')
plt.xticks(clusters)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette')
#plt.savefig(path_plots+'kmeans_silhouette.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
plt.plot(ch_score.keys(), ch_score.values(), '-o', ms=3)
elbow = KneeLocator(list(ssd_score.keys()), list(ssd_score.values()), curve='convex', direction='decreasing').knee
plt.axvline(elbow, c='tab:red', ls='--')
plt.xticks(clusters)
plt.xlabel('Number of clusters')
plt.ylabel('Calinski-Harabasz')
#plt.savefig(path_plots+'kmeans_calinski.png', bbox_inches ="tight", dpi=150)
plt.show()

#### 5.1.2 K-means with 4 clusters

In [ ]:
clf = KMeans(n_clusters=4, init='k-means++', n_init=200, algorithm='lloyd').fit(scaler.transform(X))

In [ ]:
X_scaled = scaler.transform(X)
print('Inertia: ', clf.inertia_)
print('Silhouette score: ', silhouette_score(X_scaled, clf.labels_))
print('Calinski-Harabasz score: ', calinski_harabasz_score(X_scaled, clf.labels_))

In [ ]:
# ARS: random labeling --> 0, perfect labeling --> 1
# FMS: similarity between two clustering, 0 --> no similarity, 1 --> perfect similarity

print('Adjusted Rand Score origin: ', adjusted_rand_score(y_o, clf.labels_))
print('Fowlked Mallows Score origin: ', fowlkes_mallows_score(y_o, clf.labels_))
print('Adjusted Rand Score destination: ', adjusted_rand_score(y_d, clf.labels_))
print('Fowlked Mallows Score destination: ', fowlkes_mallows_score(y_d, clf.labels_))

In [ ]:
corr_df['clust_label'] = clf.labels_

#### 5.1.3 Assign old categories to new clusters (deprecated)

In [ ]:
labels = list(set(clf.labels_))
types = ['connector', 'attractor', 'peripheral', 'local']
all_comb = []
for k in itertools.permutations(labels):
    d = {}
    for p in zip(k, types):
        d[p[0]] = p[1]
    all_comb.append(d)

In [ ]:
# origin
best_comb = {}
for c in all_comb:
    key = ('-').join([str(i) for sub in c.items() for i in sub])
    best_comb[key] = {}
    y_clust = pd.Series(clf.labels_).map(c).values
    best_comb[key]['accuracy'] = accuracy_score(y_o, y_clust)
    best_comb[key]['jaccard'] = jaccard_score(y_o, y_clust, average='weighted')
    best_comb[key]['f1'] = f1_score(y_o, y_clust, average='weighted')

In [ ]:
print('Best accuracy:', max([(k,v['accuracy']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best jaccard:', max([(k,v['jaccard']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best f1:', max([(k,v['f1']) for k,v in best_comb.items()], key=lambda x:x[1]))

In [ ]:
# destination
best_comb = {}
for c in all_comb:
    key = ('-').join([str(i) for sub in c.items() for i in sub])
    best_comb[key] = {}
    y_clust = pd.Series(clf.labels_).map(c).values
    best_comb[key]['accuracy'] = accuracy_score(y_d, y_clust)
    best_comb[key]['jaccard'] = jaccard_score(y_d, y_clust, average='weighted')
    best_comb[key]['f1'] = f1_score(y_d, y_clust, average='weighted')

In [ ]:
print('Best accuracy:', max([(k,v['accuracy']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best jaccard:', max([(k,v['jaccard']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best f1:', max([(k,v['f1']) for k,v in best_comb.items()], key=lambda x:x[1]))

In [ ]:
colors_clust = {0:'green', 1:'red', 2:'grey', 3:'orange'}

#### 5.1.4 Clustering interpretation

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(10,10))
fig.suptitle('Milano_big correlation after clustering', fontweight='bold', y=0.93)

centers = scaler.inverse_transform(clf.cluster_centers_)
colors_clust = {0:'tab:blue', 1:'tab:orange', 2:'tab:green', 3:'tab:red'}
#c = clf.labels_
c = pd.Series(clf.labels_).map(colors_clust)

axs[0,0].scatter(X[:, 0], X[:, 2], s=1, c=c)
axs[0,0].scatter(centers[:, 0], centers[:, 2], c='black', s=5)
axs[0,0].set_yscale('log')
axs[0,0].set_xlabel('k_road_o')
axs[0,0].set_ylabel('bc')

axs[0,1].scatter(X[:, 1], X[:, 2], s=1, c=c)
axs[0,1].scatter(centers[:, 1], centers[:, 2], c='black', s=5)
axs[0,1].set_yscale('log')
axs[0,1].set_xlabel('k_road_d')
axs[0,1].set_ylabel('bc')

axs[1,0].scatter(X[:, 0], X[:, 3], s=1, c=c)
axs[1,0].scatter(centers[:, 0], centers[:, 3], c='black', s=5)
axs[1,0].set_xlabel('k_road_o')
axs[1,0].set_ylabel('VOC')

axs[1,1].scatter(X[:, 1], X[:, 3], s=1, c=c)
axs[1,1].scatter(centers[:, 1], centers[:, 3], c='black', s=5)
axs[1,1].set_xlabel('k_road_d')
axs[1,1].set_ylabel('VOC')

axs[2,0].scatter(X[:, 2], X[:, 3], s=1, c=c)
axs[2,0].scatter(centers[:, 2], centers[:, 3], c='black', s=5)
axs[2,0].set_yscale('log')
axs[2,0].set_xscale('log')
axs[2,0].set_xlabel('bc')
axs[2,0].set_ylabel('VOC')

fig.delaxes(axs[2,1])

#plt.savefig(path_plots+'k_road_bc_corr.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)

In [ ]:
edge_speed_map = {}
for edge in road_network.getEdges():
    if len(edge.getOutgoing()) > 1:
        edge_speed_map[edge.getID()] = [edge.getSpeed(), 1]
    else:
        edge_speed_map[edge.getID()] = [edge.getSpeed(), 0]

In [ ]:
df_edge_speed = pd.DataFrame(edge_speed_map.items(), columns=['edge_id', 'speed'])
df_edge_speed['intersection'] = df_edge_speed['speed'].apply(lambda x: x[1])
df_edge_speed['speed'] = df_edge_speed['speed'].apply(lambda x: x[0]*3.6)

In [ ]:
road_edge_map = pd.read_csv(path_road_edge_mapping)
road_edge_map = pd.merge(df_edge_speed, road_edge_map, on='edge_id', how='left')

In [ ]:
road_edge_map = road_edge_map.groupby('road').agg({'edge_len':'sum',
                                                   'intersection':'sum',
                                                   'speed': lambda x: pd.Series.mode(x).iloc[0]}).reset_index().rename(columns={'edge_len':'road_len'})
corr_df = pd.merge(corr_df, road_edge_map, on='road', how='left')

In [ ]:
corr_df.head()

In [ ]:
# road types: high full (HF), high empty (HE), low full (LF), low empty (LE)
clust_map = {0:'HE', 1:'LE', 2:'HF', 3:'LF'}
road_clust = corr_df[['road', 'clust_label']]
road_clust['clust_label'] = road_clust['clust_label'].apply(lambda x: clust_map[x])

In [ ]:
#road_clust.to_csv(path_results+'road_clust_map.csv', index=False)

In [ ]:
centroids = pd.DataFrame(scaler.inverse_transform(clf.cluster_centers_), columns=['k_road_o', 'k_road_d', 'bc', 'voc']).reset_index().rename(columns={'index':'clust_label'})

In [ ]:
col = ['bc', 'k_road_o', 'k_road_d', 'voc', 'co2', 'road_len', 'intersection', 'speed']

fig, axs = plt.subplots(2, 4, figsize=(18,8))

for i in zip(col, itertools.product(range(2), range(4))):
    sns.boxplot(x=corr_df['clust_label'], y=corr_df[i[0]], notch=True, showcaps=False, flierprops={"marker": "x"}, ax=axs[i[1]])

axs[0,0].set_yscale('log') #bc
axs[0,3].set_yscale('log') #voc
axs[1,0].set_yscale('log') #co2
axs[1,1].set_yscale('log') #road_len

plt.show()

In [ ]:
col = ['bc', 'k_road_o', 'k_road_d', 'voc']
pp = sns.pairplot(corr_df, vars=col, kind='scatter', diag_kind='hist', hue='clust_label', palette='tab10', 
                  plot_kws={'alpha':0.7, 's':7})

for ax in pp.axes.flat:
    if ax.get_xlabel() in ['bc', 'voc']:
        ax.set(xscale='log')
    if ax.get_ylabel() in ['bc']:
        ax.set(yscale='log')

plt.show()

In [ ]:
col = ['bc', 'k_road_o', 'k_road_d', 'voc', 'co2', 'road_len', 'intersection', 'speed']

fig, axs = plt.subplots(2, 4, figsize=(18,8))

for i in zip(col, itertools.product(range(2), range(4))):
    sns.histplot(data=corr_df, x=i[0], hue='clust_label', element='step', ax=axs[i[1]], palette='tab10')

axs[0,0].set_xscale('log') #bc
axs[0,3].set_xscale('log') #voc
axs[1,0].set_xscale('log') #co2
axs[1,1].set_xscale('log') #road_len

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(4,4))

sns.scatterplot(data=corr_df, x='k_road_o', y='bc', hue='clust_label', palette='tab10', alpha = 0.6, s=10)
# plot centroids
sns.scatterplot(data=centroids, x='k_road_o', y='bc', marker='^', hue='clust_label', palette='tab10', s=70, legend=False)

# plot lines
tmp = pd.merge(corr_df, centroids, on='clust_label', how='left')
colors_clust = {0:'tab:blue', 1:'tab:orange', 2:'tab:green', 3:'tab:red'}
for idx, val in tmp.iterrows():
    x = [val['k_road_o_x'], val['k_road_o_y']]
    y = [val['bc_x'], val['bc_y']]
    plt.plot(x, y, alpha=0.2, c=colors_clust[val['clust_label']])

plt.yscale('log')

plt.show()

In [ ]:
col = ['bc', 'k_road_o', 'k_road_d', 'voc']

fig, axs = plt.subplots(2, 3, figsize=(15,8))

for i in zip(itertools.combinations(col,2), itertools.product(range(2), range(3))):
    sns.scatterplot(data=corr_df, x=i[0][0], y=i[0][1], hue='clust_label', palette='tab10', alpha = 0.7, s=7, ax=axs[i[1]])
    # plot centroids
    sns.scatterplot(data=centroids, x=i[0][0], y=i[0][1], marker='^', hue='clust_label', palette='tab10', s=70, legend=False, ax=axs[i[1]])

    # draw enclosure
    for j in corr_df['clust_label'].unique():
        points = corr_df[corr_df['clust_label'] == j][[i[0][0], i[0][1]]].values
        # get convex hull
        hull = ConvexHull(points)
        # get x and y coordinates
        # repeat last point to close the polygon
        x_hull = np.append(points[hull.vertices,0],
                        points[hull.vertices,0][0])
        y_hull = np.append(points[hull.vertices,1],
                        points[hull.vertices,1][0])
        # plot shape
        axs[i[1]].fill(x_hull, y_hull, alpha=0.1, c=colors_clust[j])

axs[0,0].set_xscale('log') #bc - k_road_o
axs[0,1].set_xscale('log') #bc - k_road_d
axs[0,2].set_xscale('log') #bc - voc
axs[0,2].set_yscale('log') #bc - voc
axs[1,1].set_yscale('log') #k_road_o - voc
axs[1,2].set_yscale('log') #k_road_d - voc

plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import graphviz

In [ ]:
X = corr_df[['k_road_o', 'k_road_d', 'bc', 'voc']].values
y = corr_df['clust_label'].values
#scaler = MinMaxScaler().fit(X)

In [ ]:
#for i in range(2, 10):
#    dt = DecisionTreeClassifier(max_depth=i, min_samples_split=50, min_samples_leaf=10)
#    print(np.mean(cross_val_score(dt, X, y, cv=10)))

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, min_samples_leaf=100)
dt = dt.fit(X, y)
print(dt.score(X, y))

In [ ]:
dot_data = export_graphviz(dt, out_file=None,
                           feature_names=['k_road_o', 'k_road_d', 'bc', 'voc'],
                            class_names=['0', '1', '2', '3'],
                           filled=True, rounded=True) 
graph = graphviz.Source(dot_data) 
graph 

In [ ]:
rf = RandomForestClassifier()
rf.fit(X, y)
print(rf.score(X, y))

feat_importances = pd.Series(rf.feature_importances_, index=['k_road_o', 'k_road_d', 'bc', 'voc'])
feat_importances.plot(kind='barh')

plt.title('Feature importance')
plt.show()

### 5.2 Hierarchical clustering

In [ ]:
# Each clustering repeated 20 times, best number of clusters selected with elbow method
## Agglomerative params     time    silhouette      calinski
## ward                     7m      [3,4]           [4,5]
## complete                 7m      [4,5]           [8,9]
## average                  6m      [6,7]           [6,7]
## single                   2m      [4,5]           [4,5]

In [ ]:
clusters = np.arange(2,11)

In [ ]:
sil_score = {} #silhouette --> 1 better, -1 worse --> elbow method
ch_score = {} #calinski_harabasz index --> the higher the better --> elbow method

In [ ]:
X_scaled = scaler.transform(X)
for i in clusters:
    sil_tmp = []
    ch_tmp = []
    for j in range(0, 20):
        clf = AgglomerativeClustering(n_clusters=i, linkage='single').fit(X_scaled)
        sil_tmp.append(silhouette_score(X_scaled, clf.labels_))
        ch_tmp.append(calinski_harabasz_score(X_scaled, clf.labels_))

    sil_score[i] = np.array(sil_tmp).mean()
    ch_score[i] = np.array(ch_tmp).mean()

In [ ]:
plt.plot(sil_score.keys(), sil_score.values(), '-o', ms=3)
elbow = KneeLocator(list(sil_score.keys()), list(sil_score.values()), curve='convex', direction='decreasing').knee
plt.axvline(elbow, c='tab:red', ls='--')
plt.xticks(clusters)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette')
plt.show()

In [ ]:
plt.plot(ch_score.keys(), ch_score.values(), '-o', ms=3)
elbow = KneeLocator(list(ch_score.keys()), list(ch_score.values()), curve='convex', direction='decreasing').knee
plt.axvline(elbow, c='tab:red', ls='--')
plt.xticks(clusters)
plt.xlabel('Number of clusters')
plt.ylabel('Calinski-Harabasz')
plt.show()

In [ ]:
clf = AgglomerativeClustering(n_clusters=4, linkage='ward').fit(scaler.transform(X))

In [ ]:
X_scaled = scaler.transform(X)
print('Silhouette score: ', silhouette_score(X_scaled, clf.labels_))
print('Calinski-Harabasz score: ', calinski_harabasz_score(X_scaled, clf.labels_))

In [ ]:
labels = list(set(clf.labels_))
types = ['connector', 'attractor', 'peripheral', 'local']
all_comb = []
for k in itertools.permutations(labels):
    d = {}
    for p in zip(k, types):
        d[p[0]] = p[1]
    all_comb.append(d)

In [ ]:
# origin
best_comb = {}
for c in all_comb:
    key = ('-').join([str(i) for sub in c.items() for i in sub])
    best_comb[key] = {}
    y_clust = pd.Series(clf.labels_).map(c).values
    best_comb[key]['accuracy'] = accuracy_score(y_o, y_clust)
    best_comb[key]['jaccard'] = jaccard_score(y_o, y_clust, average='weighted')
    best_comb[key]['f1'] = f1_score(y_o, y_clust, average='weighted')

print('Best accuracy:', max([(k,v['accuracy']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best jaccard:', max([(k,v['jaccard']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best f1:', max([(k,v['f1']) for k,v in best_comb.items()], key=lambda x:x[1]))

In [ ]:
# destination
best_comb = {}
for c in all_comb:
    key = ('-').join([str(i) for sub in c.items() for i in sub])
    best_comb[key] = {}
    y_clust = pd.Series(clf.labels_).map(c).values
    best_comb[key]['accuracy'] = accuracy_score(y_d, y_clust)
    best_comb[key]['jaccard'] = jaccard_score(y_d, y_clust, average='weighted')
    best_comb[key]['f1'] = f1_score(y_d, y_clust, average='weighted')
    
print('Best accuracy:', max([(k,v['accuracy']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best jaccard:', max([(k,v['jaccard']) for k,v in best_comb.items()], key=lambda x:x[1]))
print('Best f1:', max([(k,v['f1']) for k,v in best_comb.items()], key=lambda x:x[1]))

In [ ]:
colors_clust = {0:'grey', 1:'red', 2:'orange', 3:'green'}

In [ ]:
# ARS: random labeling --> 0, perfect labeling --> 1
# FMS: similarity between two clustering, 0 --> no similarity, 1 --> perfect similarity

print('Adjusted Rand Score origin: ', adjusted_rand_score(y_o, clf.labels_))
print('Fowlked Mallows Score origin: ', fowlkes_mallows_score(y_o, clf.labels_))
print('Adjusted Rand Score destination: ', adjusted_rand_score(y_d, clf.labels_))
print('Fowlked Mallows Score destination: ', fowlkes_mallows_score(y_d, clf.labels_))

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(10,10))
fig.suptitle('Milano_big correlation after clustering', fontweight='bold', y=0.93)

c = pd.Series(clf.labels_).map(colors_clust)

axs[0,0].scatter(X[:, 0], X[:, 2], s=1, c=c)
axs[0,0].set_yscale('log')
axs[0,0].set_xlabel('k_road_o')
axs[0,0].set_ylabel('bc')

axs[0,1].scatter(X[:, 1], X[:, 2], s=1, c=c)
axs[0,1].set_yscale('log')
axs[0,1].set_xlabel('k_road_d')
axs[0,1].set_ylabel('bc')

axs[1,0].scatter(X[:, 0], X[:, 3], s=1, c=c)
axs[1,0].set_xlabel('k_road_o')
axs[1,0].set_ylabel('VOC')

axs[1,1].scatter(X[:, 1], X[:, 3], s=1, c=c)
axs[1,1].set_xlabel('k_road_d')
axs[1,1].set_ylabel('VOC')

axs[2,0].scatter(X[:, 2], X[:, 3], s=1, c=c)
axs[2,0].set_yscale('log')
axs[2,0].set_xscale('log')
axs[2,0].set_xlabel('bc')
axs[2,0].set_ylabel('VOC')

fig.delaxes(axs[2,1])

#plt.savefig(path_plots+'k_road_bc_corr.png', bbox_inches ="tight", dpi=150)
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram

def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
plot_dendrogram(clf, truncate_mode='lastp', p=100, show_contracted=True)
plt.ylim(6700, 6900)
plt.show()

### 5.3 Plot on map

Plot on map of new clusters

In [ ]:
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, Polygon
import igraph as ig
from matplotlib.lines import Line2D

In [ ]:
road_network = sumolib.net.readNet(road_network_path, withInternal=False)
road_edge_map = pd.read_csv(path_road_edge_mapping)

G = nx.MultiDiGraph()

for edge in road_network.getEdges():
    node_from = edge.getFromNode().getID()
    node_to = edge.getToNode().getID()
    geom = [list(x) for x in edge.getShape()]
    G.add_edge(node_from, node_to, key=edge.getID(), length=edge.getLength(), geometry=LineString(geom))
    
G.graph.update({'crs': 'epsg:3857'})

In [ ]:
def plot_df(df_exps, road_edge_map, road_net):
    road_edge_map_no_intern = road_edge_map[~road_edge_map['edge_id'].astype(str).str.startswith(':')]
    road_edge_mean_map = pd.merge(road_edge_map_no_intern, df_exps, on=['road'])
    
    # create column with tuple of edges of the graph (u,v,key)
    edge_graph_list = []
    for edge in road_edge_mean_map['edge_id']:
        from_node = road_net.getEdge(edge).getFromNode().getID()
        to_node = road_net.getEdge(edge).getToNode().getID()
        edge_graph_list.append((from_node, to_node, edge))
        
    road_edge_mean_map['edge_graph'] = edge_graph_list
    
    return road_edge_mean_map

In [ ]:
corr_plot = plot_df(corr_df, road_edge_map, road_network)

In [ ]:
corr_plot.head()

In [ ]:
def add_attribute_to_graph(graph, df_plot, attr_name, plot_col):
    # edge[0] = node_from, edge[1] = node_to, edge[2] = key = edge_id 
    
    # Initialize co2 attribute in the graph
    for edge in graph.edges:
        G[edge[0]][edge[1]][edge[2]][attr_name] = None
        
    # Set co2 attribute based on some value per road
    for edge, value in zip(df_plot['edge_graph'], df_plot[plot_col]):
        graph[edge[0]][edge[1]][edge[2]][attr_name] = value

In [ ]:
colors = {0:'orange', 1:'grey', 2:'red', 3:'green'}
#colors = {0:'tab:blue', 1:'tab:orange', 2:'tab:green', 3:'tab:red'}
corr_plot['color_clust'] = corr_plot['clust_label'].map(colors)

In [ ]:
add_attribute_to_graph(G, corr_plot, 'color_clust', 'color_clust')

In [ ]:
# create colormap

val = []
ind = []
for edge in G.edges:
    if G[edge[0]][edge[1]][edge[2]]['color_clust'] == None:
        val.append('white')
        ind.append(edge)
    else:
        val.append(G[edge[0]][edge[1]][edge[2]]['color_clust'])
        ind.append(edge)  

ec = pd.Series(val, index=pd.MultiIndex.from_tuples(ind))

In [ ]:
fig, ax = ox.plot_graph(G, bgcolor='lightgrey', node_size=0, edge_linewidth=1, edge_color=ec, show=False)

plt.title('Milano-big: type of road')

legend_elements = [Line2D([0], [0], color='red', lw=4, label='High k_road, high voc'),
                   Line2D([0], [0], color='orange', lw=4, label='High k_road, low voc'),
                   Line2D([0], [0], color='green', lw=4, label='Low k_road, high voc'),
                   Line2D([0], [0], color='grey', lw=4, label='Low k_road, low voc')
                   ]

ax.legend(handles=legend_elements, bbox_to_anchor=(1.2, 1))
plt.savefig(path_plots+'corr_map_clustering.png', bbox_inches ="tight")
plt.show()

## 6. CO2 prediction

Code for the prediction of the CO2 emissions using the clustering obtained from the K-means.

In [ ]:
# Results for CO2 prediction by different feature sets
## Features                 Best model      Train R^2       Test R^2
## 4 original features      ElasticNet      0.43            0.57           
## No VOC                   ElasticNet      0.27            0.31
## Capacity instead of VOC  ElasticNet      0.31            0.24
## Extra features           ElasticNet      0.30            0.29

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [ ]:
# Add capacity of roads
road_network = sumolib.net.readNet(road_network_path, withInternal=False)
df_edge_capacity = compute_edge_capacity(road_network.getEdges())
df_edge_capacity = pd.DataFrame(df_edge_capacity.items(), columns=['edge_id', 'capacity'])
road_edge_map = pd.read_csv(path_road_edge_mapping)
df_edge_capacity = pd.merge(df_edge_capacity, road_edge_map, on='edge_id', how='left')
weighted_avg = lambda x: np.average(x, weights=df_edge_capacity.loc[x.index, "edge_len"])
df_edge_capacity = df_edge_capacity.groupby(by=['road']).agg({'capacity': weighted_avg}).reset_index()
corr_df = pd.merge(corr_df, df_edge_capacity, on='road')

In [ ]:
corr_df2 = corr_df.sample(frac=1)

#X = corr_df2[['k_road_o', 'k_road_d', 'bc', 'voc']]
#X = corr_df2[['k_road_o', 'k_road_d', 'bc']]
#X = corr_df2[['k_road_o', 'k_road_d', 'bc', 'capacity']]
X = corr_df2[['k_road_o', 'k_road_d', 'bc', 'capacity', 'road_len', 'intersection', 'speed']]
y = corr_df2['co2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
parameters = [{
                'model': [LinearRegression()],
                'model__fit_intercept': [True],
                #'model__positive': [True, False],
              },
              {
                'model': [Ridge()],
                #'model__alpha': [0.1, 1, 5, 10, 25, 50, 100],
                'model__alpha': np.arange(1,20,0.1),
                'model__fit_intercept': [True],
                #'model__positive': [True, False],
                #'model__solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
              },
              {
                'model': [Lasso()],
                #'model__alpha': [0.1, 1, 5, 10, 25, 50, 100],
                'model__alpha': np.arange(1,20,0.1),
                'model__fit_intercept': [True],
                #'model__positive': [True, False],
                #'model__selection': ['cyclic', 'random'],
                #'model__warm_start': [True, False]
              },
              {
                'model': [ElasticNet()],
                #'model__alpha': [0.1, 1, 5, 10, 25, 50, 100],
                'model__alpha': np.arange(0.1,3,0.05),
                'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                'model__fit_intercept': [True],
                #'model__positive': [True, False],
                #'model__selection': ['cyclic', 'random'],
                #'model__warm_start': [True, False]
              }
            ]

pipe = Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())])
clf = GridSearchCV(pipe, param_grid=parameters, cv=5, refit=True, scoring='r2')
clf.fit(X_train, y_train)

In [ ]:
print('Best model: ', clf.best_params_['model'])
print('Training score: ', clf.best_score_)
model = clf.best_estimator_
y_pred = model.predict(X_test)
print('Test score: ', r2_score(y_test, y_pred))

In [ ]:
col = ['k_road_o', 'k_road_d', 'bc', 'voc']
#col = ['k_road_o', 'k_road_d', 'bc']
#col = ['k_road_o', 'k_road_d', 'bc', 'capacity']
#col = ['k_road_o', 'k_road_d', 'bc', 'capacity', 'road_len', 'intersection', 'speed']

fig, axs = plt.subplots(2, 2, figsize=(12,7))

for i in zip(col, itertools.product(range(2), range(2))):
    sns.regplot(x=X_test[i[0]], y=y_pred, scatter_kws={'s':5, 'alpha':0.5}, label='Predicted', ax=axs[i[1]])
    sns.regplot(x=X_test[i[0]], y=y_test, scatter_kws={'s':5, 'alpha':0.5}, label='Observed', ax=axs[i[1]])
    axs[i[1]].set_yscale('log')
    axs[i[1]].legend()

axs[1,0].set_xscale('log')
plt.show()